# 1. Introduction to the Python API

An advantage of the Python API is that you can use it in your own development environment (vs. the JS Code Editor) - so you get a lot more flexibility to automate as well as combine other analysis and visualization libraries with Earth Engine.

## Installation

The preferred method for installing the Earth Engine Python API is via Anaconda. Follow these steps to install Anaconda and required libraries.

1. Download the [Anaconda Installer](https://www.anaconda.com/products/individual) for Python 3.7 (or a higher version) for your operating system. Once downloaded, double click the installer and install it into the default suggested directory. Select an install for "Just Me" and use default settings. Note: If your username has spaces, or non-English characters, it may cause problems. In that case, you can install it to a path such as C:\anaconda.

<div style="align:center;">
    <img style="width:66%" src="https://courses.spatialthoughts.com/images/end_to_end_gee/conda1.png" />
</div>

2. (Windows users) Once installed, search for Anaconda Prompt launch it. (Mac/Linux users): Launch a Terminal window.

<div style="align:center;">
    <img style="width:66%" src="https://courses.spatialthoughts.com/images/end_to_end_gee/conda2.png" />
</div>

3. Once the prompt launches, it will be in the base environment. It is a good practice to install new packages in a separate environment. We will new create a new environment to install Earth Engine related packages. Enter the below code to create a new environment called ee.

> Note: You can use the shortcut Shift + Insert to paste commands in Anaconda Prompt.

```
conda create --name ee
```

<div style="align:center;">
    <img style="width:66%" src="https://courses.spatialthoughts.com/images/end_to_end_gee/conda3.png" />
</div>

4. Anaconda will validate the conda commands and before proceeding will ask for confirmation, press y, and enter to confirm.

<div style="align:center;">
    <img style="width:66%" src="https://courses.spatialthoughts.com/images/end_to_end_gee/conda4.png" />
</div>

5. Once the environment is created the Executing transaction: done message will be displayed, if the execution is not successful, it will be marked as failed with an error message. Enter the code to activate the environment.

```
conda activate ee
```

<div style="align:center;">
    <img style="width:66%" src="https://courses.spatialthoughts.com/images/end_to_end_gee/conda5.png" />
</div>

6. Now the (base) will be replaced with (ee) indicating the environment is activated. Libraries in the `ee` environment will not affect the base environment. Also, these libraries can be used only in this environment cannot be accessed from the base or other environments.

<div style="align:center;">
    <img style="width:66%" src="https://courses.spatialthoughts.com/images/end_to_end_gee/conda6.png" />
</div>

7. Now, `earthengine-api` library can be installed using `conda-forge`. When entering the below code, anaconda prompt will ask for confirmation, press y and Enter to install the library.

```conda install -c conda-forge earthengine-api```

<div style="align:center;">
    <img style="width:66%" src="https://courses.spatialthoughts.com/images/end_to_end_gee/conda7.png" />
</div>

8. On successful installation, `Executing transaction: done`, will be displayed, enter the below code to install geemap library.

```
conda install geopandas
conda install mamba -c conda-forge
mamba install geemap xarray_leaflet -c conda-forge
```

9. Once the library installs successfully, enter the below code to install jupyter lab, it is a browser-based user interface that can be used for executing python commands in a notebook format file.

```mamba install -c conda-forge jupyterlab```

<div style="align:center;">
    <img style="width:66%" src="https://courses.spatialthoughts.com/images/end_to_end_gee/conda9.png" />
</div>

10. On successful installation Executing transaction: done will be displayed. To launch it enter the below code and click enter.

```jupyter lab```

<div style="align:center;">
    <img style="width:66%" src="https://courses.spatialthoughts.com/images/end_to_end_gee/conda10.png" />
</div>

11. A new browser tab will open with an instance of JupterLab. Click the Python 3 button under Notebooks.

> Note: Do not close the Anaconda prompt as the jupyter server is running using this connection, which will be interrupted if the prompt is closed.

<div style="align:center;">
    <img style="width:66%" src="https://courses.spatialthoughts.com/images/end_to_end_gee/conda11.png" />
</div>

12. Enter the below code and click the Run button, if the execution is successful all the libraries and their dependencies are installed successfully.

```
import ee
import geemap
```

<div style="align:center;">
    <img style="width:66%" src="https://courses.spatialthoughts.com/images/end_to_end_gee/conda12.png" />
</div>

## Python API Syntax

Coming from Earth Engine programming through the Code Editor, you will need to slightly adapt your scripts to be able to run in Python. For the bulk of your code, you will be using Earth Engine API’s server-side objects and functions - which will be exactly the same in Python. You only need to make a few syntactical changes.

Here’s the full list of [differences](https://developers.google.com/earth-engine/guides/python_install#syntax). Most important ones are elaborated below:

### Initialization

First of all, you need to run the following cells to initialize the API and authorize your account. You will be prompted to sign-in to the account and allow access to View and Manage your Earth Engine data. 

Once you approve, you will get an a verification code that needs to be entered at the prompt. This step needs to be done just once per session.

In [1]:
import ee
# ee.Authenticate()  # Only run for the first time.
ee.Initialize()

import geemap

### Variables

In [2]:
city = 'San Fransico'
state = 'California'
print(city, state)

population = 881549
print(population)

San Fransico California
881549


### Line Continuation

In [4]:
s2 = ee.ImageCollection('COPERNICUS/S2')
filtered = s2\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))\
    .filter(ee.Filter.date('2019-01-01', '2019-12-31'))

### Functions

In [5]:
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
        qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

### Function Arguments

Named arguments to Earth Engine functions need to be in quotes. Also when passing the named arguments as a dictionary, it needs to be passed using the `**` keyword.

In [6]:
gaul = ee.FeatureCollection("FAO/GAUL/2015/level1")
gfsad = ee.Image("USGS/GFSAD1000_V0")
wheatrice = gfsad.select('landcover').eq(1)
uttarpradesh = gaul.filter(ee.Filter.eq('ADM1_NAME', 'Uttar Pradesh'))
points = wheatrice.selfMask().stratifiedSample(**{
    'numPoints': 100, 
    'region': uttarpradesh, 
    'geometries': True
})
print(points.getInfo())

{'type': 'FeatureCollection', 'columns': {'landcover': 'Byte<0, 1>'}, 'properties': {'band_order': ['landcover']}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [80.54859931087773, 26.557191000167748]}, 'id': '0', 'properties': {'landcover': 1}}, {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [78.14691430281763, 28.164266470245117]}, 'id': '1', 'properties': {'landcover': 1}}, {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [81.18250130185268, 26.869677897127232]}, 'id': '2', 'properties': {'landcover': 1}}, {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [82.54851545141847, 27.15538020291877]}, 'id': '3', 'properties': {'landcover': 1}}, {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [79.93255371401474, 26.887534291239206]}, 'id': '4', 'properties': {'landcover': 1}}, {'type': 'Feature', 'ge

### In-line functions

The syntax for defining in-line functions is also slightly different. You need to use the lambda keyword:

In [7]:
points = points.map(lambda feature: ee.Feature(feature.geometry(), {'id': feature.id()} ))

### Printing Values

The `print()` function syntax is the same. But you must remember that in the Code Editor, when you can print, the value of the server object is fetched and then printed. You must do that explicitely by calling `getInfo()` on any server-side object:

In [8]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [80.54859931087773, 26.557191000167748]}, 'id': '0', 'properties': {'id': '0'}}


### Automatic Conversion of Scripts

[geemap](https://github.com/giswqs/geemap) is an open-source Python package that comes with many helpful features that help you use Earth Engine effectively in Python.

It comes with a function that can help you translate your javascript earth engine code to Python automatically.

The automatic conversion works great, but it is not perfect. You may have to tweak the output a bit. An important filter that is missing from the Python API is the `ee.Filter.bounds()`. You can use an alternative `ee.Filter.geometry()` instead.

In [9]:
javascript_code = """
var geometry = ee.Geometry.Point([107.61303468448624, 12.130969369851766]);
Map.centerObject(geometry, 12)
var s2 = ee.ImageCollection("COPERNICUS/S2")
var rgbVis = {
  min: 0.0,
  max: 3000,
  bands: ['B4', 'B3', 'B2'],
};

// Write a function for Cloud masking
function maskS2clouds(image) {
  var qa = image.select('QA60')
  var cloudBitMask = 1 << 10;
  var cirrusBitMask = 1 << 11;
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0).and(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask)
      .select("B.*")
      .copyProperties(image, ["system:time_start"])
}
 
var filtered = s2
  .filter(ee.Filter.date('2019-01-01', '2019-12-31'))
  .filter(ee.Filter.bounds(geometry))
  .map(maskS2clouds)
  

// Write a function that computes NDVI for an image and adds it as a band
function addNDVI(image) {
  var ndvi = image.normalizedDifference(['B5', 'B4']).rename('ndvi');
  return image.addBands(ndvi);
}

var withNdvi = filtered.map(addNDVI);

var composite = withNdvi.median()
palette = [
  'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301'];

ndviVis = {min:0, max:0.5, palette: palette }
Map.addLayer(withNdvi.select('ndvi'), ndviVis, 'NDVI Composite')

"""

In [13]:
geemap.js_snippet_to_py(javascript_code)

In [16]:
import ee
import geemap

Map = geemap.Map()

geometry = ee.Geometry.Point([107.61303468448624, 12.130969369851766])
Map.centerObject(geometry, 12)
s2 = ee.ImageCollection("COPERNICUS/S2")
rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

# Write a function for Cloud masking
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

filtered = s2 \
  .filter(ee.Filter.date('2019-01-01', '2019-12-31')) \
  .filter(ee.Filter.geometry(geometry)) \
  .map(maskS2clouds)

# Write a function that computes NDVI for an image and adds it as a band
def addNDVI(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('ndvi')
    return image.addBands(ndvi)

withNdvi = filtered.map(addNDVI)

composite = withNdvi.median()
palette = [
  'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301']

ndviVis = {'min':0, 'max':0.5, 'palette': palette }
Map.addLayer(withNdvi.select('ndvi'), ndviVis, 'NDVI Composite')
Map

Map(center=[12.130969369851766, 107.61303468448625], controls=(WidgetControl(options=['position', 'transparent…

---